In [ ]:
import torch
from torchaudio.models.decoder import ctc_decoder
import os
import torchaudio
from model import Model

In [9]:
char_to_int = {
    ' ': 0,
    "'": 1,
    'a': 2,
    'b': 3,
    'c': 4,
    'd': 5,
    'e': 6,
}
string = "abcde"
int_list = [char_to_int[char]+char_to_int['\''] for char in string]

print(int_list)
result = "-"+"-".join(string) + "-"
print(result)

[3, 4, 5, 6, 7]
-a-b-c-d-e-


In [ ]:
folder_dir = os.path.join(os.getcwd(), 'models')
lexicon_file = "lexicon.txt"
lexicon_path = os.path.join(folder_dir, lexicon_file)
lm_file = "lm.bin"
lm_path = os.path.join(folder_dir, lm_file)
acoustic_file = "states.pth"
acoustic_path = os.path.join(folder_dir, acoustic_file)
labels = [
    " ",
    *"abcdefghijklmnopqrstuvwxyz",
    "'",
    "*"
]
tokens = ['-', '|', 'e', 't', 'a', 'o', 'n', 'i', 'h', 's', 'r', 'd', 'l', 'u',
          'm', 'w', 'c', 'f', 'g', 'y', 'p', 'b', 'v', 'k', "'", 'x', 'j', 'q', 'z']

In [ ]:
bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()
torch.save(model,'base.pt')

In [ ]:
# model = torch.load('models/weights.pt')
model = torch.load('models/base.pt')
acoustic_model = Model(num_classes=len(labels))
acoustic_model.load_state_dict(torch.load(acoustic_path))

In [ ]:
LM_WEIGHT = 3.23
WORD_SCORE = -0.26
blank_token = '*'
sil_token = ' '
my_decoder = ctc_decoder(
    lexicon=lexicon_path,
    tokens=labels,
    lm=lm_path,
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
    blank_token=blank_token,
    sil_token=sil_token
)
ctc_decode = ctc_decoder(
    lexicon=lexicon_path,
    tokens=tokens,
    lm=lm_path,
    beam_size_token=len(tokens),
    lm_weight=LM_WEIGHT,
    word_score=WORD_SCORE,
)

In [4]:
import sounddevice as sd
import soundfile as sf


def record_audio(duration, sample_rate=16000, file_path="test.wav"):
    # Set the sample rate and duration
    duration_in_samples = int(duration * sample_rate)

    # Record audio
    audio = sd.rec(frames=duration_in_samples,
                   samplerate=sample_rate, channels=1, blocking=True)

    # Save audio to file
    sf.write(file_path, audio, sample_rate)

    return file_path


# Example usage: Record audio for 5 seconds at a sample rate of 16000 Hz and save to "recorded_audio.wav"
audio_file = record_audio(duration=3, sample_rate=16000,
                          file_path="test.wav")
print("Audio saved to:", audio_file)

Audio saved to: test.wav


In [ ]:
waveform, sample_rate = torchaudio.load(audio_file)
print("Sample rate:", sample_rate)

In [ ]:
model.eval()
with torch.no_grad():
    emission, _ = model(waveform)

In [ ]:
from main import transcribe

In [ ]:
print(transcribe(audio_file, acoustic_model, my_decoder))

In [ ]:
beam_search_result = ctc_decode(emission)
beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()

In [ ]:
print(f"Transcript: {beam_search_transcript}")

In [1]:
import main
import utils

c:\Program Files\Python39\lib\site-packages\torchaudio\models\decoder\_ctc_decoder.py:62: UserWarning: The built-in flashlight integration is deprecated, and will be removed in future release. Please install flashlight-text. https://pypi.org/project/flashlight-text/ For the detail of CTC decoder migration, please see https://github.com/pytorch/audio/issues/3088.
  warnings.warn(


In [2]:
model, decoder = utils.load_files_mod()

All files downloaded


In [5]:
print(main.transcribe_mod(audio_file, model, decoder))

this is a beautiful girl
